# HW #2: Neural Networks

[change this (hw #1)]
This experiment uses a neural network of ten Networks encapsulated in a Network class object.
The Networks themselves exist in a weight matrix property of the Network class that is returned
as a Numpy array from the train() method.  There are two layers in this network.
An input layer represented as xᴷ of 784 inputs, plus one for the bias.  And an output layer
represented as yᴷ. There are no hidden layers in this experiment.  We ran the experiment with learning
rates, represented as η, of 0.00001, 0.001, and 0.1.

     xᴷ      The input layer vector of 784 (+1 bias) pixels of a single image
     yᴷ      The output (activation) layer vector of 10 nodes representing activation level of a digit
     tᴷ      The output layer vector of 10 nodes from the labeled training data
     wᵢ      The weight element at index i of 785 weights
     Δwᵢ     The gradient or vector derivative in order to minimize the cost function

The Network Learning Algorithm was implemented by forwarding an activation vector of 10 elements
to the output layer in the forward() method.


      yᴷ = a(w · x)

After forwarding the activation layer, we updated weights through back propagation using
the following formula in the back() method:

     Δwᵢ = η(tᴷ - yᴷ)xᵢᴷ
      wᵢ ⟵ wᵢ + Δwᵢ


## Experiment 1

In [ ]:
import network as nn

First we instantiate our Network object (or rather network of 10 Networks).
We set the bias, the number of epochs and the sizes of our input and output layers.

The Network constructor optionally takes in the training and test file names and loads them
into Numpy arrays.

Here are the parameters that actually will remain static for all of our experiments.
In fact, aside from the filenames for the training and validation samples, they are the
the default parameter values for training.

In [ ]:
train_file = 'mnist_train.csv'
test_file = 'mnist_validation.csv'

bias = 1
epochs = 50
rate = 0.1
target = 0.9
initial_weight = 0.05

Here are the the parameters that will actually change during the experiments:

#### Hidden Input Size
This is number of nodes in the hidden layer.  Conceptually, this represents the number of
intermediate things (or abstractions) that this network will learn about hand written digits.
Naively, we might be tempted to think of this as different shapes we find in a handwritten image.
For example, both a 9 and an 8 have a circle at the upper have of the image. Both a 5 and a 3,
have a semicircle in the lower half of an image.  There are many shapes that the digits below share.

123456789

However,
The one parameter that will change in this entire projec is the momentum, represented by α in the lecture slides.

In [ ]:
input_size = 785
hidden_size = 20
output_size = 10
sizes = [input_size, hidden_size, output_size]

momentum = 0.9

### Train with a hidden layer size of 20 nodes

In [ ]:
input_size = 785
hidden_size = 20
output_size = 10

sizes = [input_size, hidden_size, output_size]

n = nn.Network(sizes=sizes, train_filename=train_file, test_filename=test_file, bias=bias)
wᵢ, wⱼ, accuracy = n.train(η=rate, epochs=epochs)
assert(wᵢ.shape == (785, 20))
assert(wⱼ.shape == (20 + bias, 10))
assert(accuracy > .90)

### Train with a hidden layer size of 50 nodes

In [ ]:
hidden_size = 50
sizes = [input_size, hidden_size, output_size]
n.resize(sizes)
wᵢ, wⱼ, accuracy = n.train(epochs=epochs)
assert(accuracy > .90)

### Train with a hidden layer size of 100 nodes

In [ ]:
hidden_size = 100
sizes = [input_size, hidden_size, output_size]
n.resize(sizes)
wᵢ, wⱼ, accuracy = n.train(epochs=epochs)
assert(accuracy > .90)

## Summary

The digits having the most issues being classified were 8, 5, and 2, all being mistaken for a 3.
The rest of the digits did quite well, especially 0, 1, 5, and 7.  That plots and confusions
matrices do not show any significant differrences between our three learning rates.

I noticed with all them that starting with the Epoch 0 the accuracy was very low (as expected).
It was less than 10%.  At Epoch 1, the accuracy jumps to ~85% and then over the next 49 epochs
we only get an improvement of 3%.  I am confused by this.  Also, I don't see any oscillations
here, so it appears that I am overfitting.

